# Nightly Report: IQ, AOS, Thermal, Aberrations and Degrees of Freedom

Owner: **Guillem Megias** <br>
Last Verified to Run: **2025-07-07** <br>

In [ ]:
# Times Square Parameters
day_obs = 20251124
seq_min = 0
seq_max = 900

# This version has added back the bending modes.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
from matplotlib.patches import Rectangle

%matplotlib inline

In [ ]:
import galsim
import numpy as np
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)

def getPsfGradPerZernike(
    diameter: float = 8.36,
    obscuration: float = 0.612,
    jmin: int = 4,
    jmax: int = 22,
) -> np.ndarray:
    """Get the gradient of the PSF FWHM with respect to each Zernike.

    This function takes no positional arguments. All parameters must be passed
    by name (see the list of parameters below).

    Parameters
    ----------
    diameter : float, optional
        The diameter of the telescope aperture, in meters.
        (the default, 8.36, corresponds to the LSST primary mirror)
    obscuration : float, optional
        Central obscuration of telescope aperture (i.e. R_outer / R_inner).
        (the default, 0.612, corresponds to the LSST primary mirror)
    jmin : int, optional
        The minimum Noll index, inclusive. Must be >= 0. (the default is 4)
    jmax : int, optional
        The max Zernike Noll index, inclusive. Must be >= jmin.
        (the default is 22.)

    Returns
    -------
    np.ndarray
        Gradient of the PSF FWHM with respect to the corresponding Zernike.
        Units are arcsec / micron.

    Raises
    ------
    ValueError
        If jmin is negative or jmax is less than jmin
    """
    # Check jmin and jmax
    if jmin < 0:
        raise ValueError("jmin cannot be negative.")
    if jmax < jmin:
        raise ValueError("jmax must be greater than jmin.")

    # Calculate the conversion factors
    conversion_factors = np.zeros(jmax + 1)
    for i in range(jmin, jmax + 1):
        # Set coefficients for this Noll index: coefs = [0, 0, ..., 1]
        # Note the first coefficient is Noll index 0, which does not exist and
        # is therefore always ignored by galsim
        coefs = [0] * i + [1]

        # Create the Zernike polynomial with these coefficients
        R_outer = diameter / 2
        R_inner = R_outer * obscuration
        Z = galsim.zernike.Zernike(coefs, R_outer=R_outer, R_inner=R_inner)

        # We can calculate the size of the PSF from the RMS of the gradient of
        # the wavefront. The gradient of the wavefront perturbs photon paths.
        # The RMS quantifies the size of the collective perturbation.
        # If we expand the wavefront gradient in another series of Zernike
        # polynomials, we can exploit the orthonormality of the Zernikes to
        # calculate the RMS from the Zernike coefficients.
        rms_tilt = np.sqrt(np.sum(Z.gradX.coef**2 + Z.gradY.coef**2) / 2)

        # Convert to arcsec per micron
        rms_tilt = np.rad2deg(rms_tilt * 1e-6) * 3600

        # Convert rms -> fwhm
        fwhm_tilt = 2 * np.sqrt(2 * np.log(2)) * rms_tilt

        # Save this conversion factor
        conversion_factors[i] = fwhm_tilt

    return conversion_factors[jmin:]


def convertZernikesToPsfWidth(
    zernikes: np.ndarray,
    diameter: float = 8.36,
    obscuration: float = 0.612,
    jmin: int = 4,
) -> np.ndarray:
    """Convert Zernike amplitudes to quadrature contribution to the PSF FWHM.

    Parameters
    ----------
    zernikes : np.ndarray
        Zernike amplitudes (in microns), starting with Noll index `jmin`.
        Either a 1D array of zernike amplitudes, or a 2D array, where each row
        corresponds to a different set of amplitudes.
    diameter : float
        The diameter of the telescope aperture, in meters.
        (the default, 8.36, corresponds to the LSST primary mirror)
    obscuration : float
        Central obscuration of telescope aperture (i.e. R_outer / R_inner).
        (the default, 0.612, corresponds to the LSST primary mirror)
    jmin : int
        The minimum Zernike Noll index, inclusive. Must be >= 0. The
        max Noll index is inferred from `jmin` and the length of `zernikes`.
        (the default is 4, which ignores piston, x & y offsets, and tilt.)

    Returns
    -------
    dFWHM: np.ndarray
        Quadrature contribution of each Zernike vector to the PSF FWHM
        (in arcseconds).

    Notes
    -----
    Converting Zernike amplitudes to their quadrature contributions to the PSF
    FWHM allows for easier physical interpretation of Zernike amplitudes and
    the performance of the AOS system.

    For example, image we have a true set of zernikes, [Z4, Z5, Z6], such that
    ConvertZernikesToPsfWidth([Z4, Z5, Z6]) = [0.1, -0.2, 0.3] arcsecs.
    These Zernike perturbations increase the PSF FWHM by
    sqrt[(0.1)^2 + (-0.2)^2 + (0.3)^2] ~ 0.37 arcsecs.

    If the AOS perfectly corrects for these perturbations, the PSF FWHM will
    not increase in size. However, imagine the AOS estimates zernikes, such
    that ConvertZernikesToPsfWidth([Z4, Z5, Z6]) = [0.1, -0.3, 0.4] arcsecs.
    These estimated Zernikes, do not exactly match the true Zernikes above.
    Therefore, the post-correction PSF will still be degraded with respect to
    the optimal PSF. In particular, the PSF FWHM will be increased by
    sqrt[(0.1 - 0.1)^2 + (-0.2 - (-0.3))^2 + (0.3 - 0.4)^2] ~ 0.14 arcsecs.

    This conversion depends on a linear approximation that begins to break down
    for RSS(dFWHM) > 0.20 arcsecs. Beyond this point, the approximation tends
    to overestimate the PSF degradation. In other words, if
    sqrt(sum( dFWHM^2 )) > 0.20 arcsec, it is likely that dFWHM is
    over-estimated. However, the point beyond which this breakdown begins
    (and whether the approximation over- or under-estimates dFWHM) can change,
    depending on which Zernikes have large amplitudes. In general, if you have
    large Zernike amplitudes, proceed with caution!
    Note that if the amplitudes Z_est and Z_true are large, this is okay, as
    long as |Z_est - Z_true| is small.

    For a notebook demonstrating where the approximation breaks down:
    https://gist.github.com/jfcrenshaw/24056516cfa3ce0237e39507674a43e1

    Raises
    ------
    ValueError
        If jmin is negative
    """
    # Check jmin
    if jmin < 0:
        raise ValueError("jmin cannot be negative.")

    # Calculate jmax from jmin and the length of the zernike array
    jmax = jmin + np.array(zernikes).shape[-1] - 1

    # Calculate the conversion factors for each zernike
    conversion_factors = getPsfGradPerZernike(
        jmin=jmin,
        jmax=jmax,
        diameter=diameter,
        obscuration=obscuration,
    )

    # Convert the Zernike amplitudes from microns to their quadrature
    # contribution to the PSF FWHM
    dFWHM = conversion_factors * zernikes

    return dFWHM

band_colors = {
    "u": "#0c71ff",
    "g": "#49be61",
    "r": "#c61c00",
    "i": "#ffc200",
    "z": "#f341a2",
    "y": "#5d0000",
}
    
def annotate_bands(data: pd.DataFrame, ax: plt.Axes) -> None:
    """Anotate bottom of plot with band colors

    Parameters
    ----------
    data : pd.DataFrame
        Table of data that is being plotted
    ax : plt.Axes
        Axis on which to add annotations
    """
    # Get the bands
    bands = data['band']

    # Get the axis limits
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    # Plot band bars at bottom of plot
    max_seq = data.index[-1]
    for i, band in enumerate(bands[:-1]):
        ax.plot(
            data.index[i : i + 2],
            2 * [ylim[0]],
            c=band_colors[band[0]],
            lw=7,
        )
    # Do something special for last one
    ax.plot(
        [data.index[-1], data.index[-1]+1],
        2 * [ylim[0]],
        c=band_colors[bands.iloc[-1][0]],
        lw=7,
    )

    # Restore the axis limits
    ax.set_ylim(ylim)
    ax.set_xlim(xlim)

    # Add unique bands to legend
    unique_bands = data['band'].str[0].unique()  # Get first character of each band
    
    for band_char in sorted(unique_bands):
        ax.scatter([], [], 
                  c=band_colors[band_char], 
                  s=100, 
                  marker='s', 
                  label=f'{band_char}')

def find_blocks(data: pd.DataFrame) -> pd.DataFrame:
    """Find blocks active during the night

    Parameters
    ----------
    data : pd.DataFrame
        Table of data that is being plotted
    Returns
    -------
    changes: pd.DataFrame
        dataframe containing the sequence numbers where the block changed
        and the name of the new block
    """
    block_data = data[['seq', 'block']].sort_values(by='seq')
    change_mask = block_data['block'] != block_data['block'].shift()
    block_data['block'] = block_data['block'].str.replace('BLOCK-', '', regex=False)
    blocks = pd.DataFrame({
        'seq': block_data.loc[change_mask, 'seq'],
        'block_after': block_data['block'].loc[change_mask]
    })
    return blocks

async def find_faults(client, table):
    """Find faults during the night

    Parameters
    ----------
    client: EFD client
    table : pd.DataFrame
        Table of data that is being plotted
    Returns
    -------
    table: pd.DataFrame
        incoming dataframe with the fault events added
    """
    table_time = pd.to_datetime(table['time'], format='ISO8601', utc=True)
    topics = ['MTMount', 'MTAOS', 'MTHexapod', 'MTCamera']
    table['faults'] = [None for i in range(len(table))]
    start = Time(table['obs_start'].iloc[0])
    end = Time(table['obs_end'].iloc[-1])
    for topic in topics:
        efd_data = await client.select_time_series(f"lsst.sal.{topic}.logevent_summaryState", \
                                                ['summaryState'], \
                                                 start, end)
        if len(efd_data) == 0:
            continue
        faults = efd_data[efd_data['summaryState'] == 3]
        for i in range(len(faults)):
            fault_time = pd.to_datetime(faults.index[i], format='utc')
            closest_row = table.iloc[(table_time - fault_time).abs().argmin()]
            table.loc[table['seq'] == closest_row['seq'], 'faults'] = topic
    return table


In [ ]:
from lsst.ts.xml.tables.m1m3 import *
from lsst.ts.m1m3.utils import *


async def get_m1m3_gradients(client, data):
    """Get the M1M3 thermal gradients

    Parameters
    ----------
    client: EFD client
    data : pd.DataFrame
        Table of minimal data
    Returns
    -------
    data: pd.DataFrame
        incoming dataframe with the m1m3 temperature gradients added
    """
    data_times = pd.to_datetime(data['obs_start'], format='ISO8601', utc=True)
    sorted_data_times = data_times.sort_values()
    start = Time(sorted_data_times.iloc[0])
    end = Time(sorted_data_times.iloc[-1])
    data_times = data_times.astype('int64')
    thermocouples = ThermocoupleAnalysis(client)
    await thermocouples.load(start, end, time_bin=30)
    gradients = thermocouples.xyz_r_gradients
    grad_times = pd.to_datetime(gradients.index, format='ISO8601', utc=True).astype('int64')
    t0 = grad_times[0]
    grad_times -= t0
    grad_times /=1E9
    data_times -= t0
    data_times /= 1E9
    names = ['x_gradient', 'y_gradient', 'z_gradient', 'radial_gradient']
    for name in names:
        values = gradients[name].values
        val_series = pd.Series(values)
        val_interpolated = val_series.interpolate()
        data[name] = np.interp(data_times, grad_times, val_interpolated)
    return data


In [ ]:
from matplotlib.patches import Patch
from matplotlib import cm

def plot_m1m3_gradients(data, ax):
    """Plot the M1M3 thermal gradients

    Parameters
    ----------
    data : pd.DataFrame
        Table of data to plot
    ax : matplotlib.axes._axes.Axes
        axes object on which to plot the data
    Returns
    -------
    """

    # --- Palettes: same hue family within groups, distinct shades per line ---
    blues   = plt.get_cmap('Blues')
    oranges = plt.get_cmap('Oranges')
    
    # Line colors (distinct, but clearly grouped)
    c_x = blues(0.75)     # darker blue
    c_y = blues(0.55)     # lighter blue
    c_r = oranges(0.75)   # darker orange
    c_z = oranges(0.55)   # lighter orange
    
    # Band colors (very light tint of the group hue)
    band_xy = blues(0.25)
    band_rz = oranges(0.25)
    
    # --- Shaded operational bands (put behind data) ---
    ax.axhspan(-0.4, 0.4, facecolor=band_xy, alpha=0.3, zorder=0)
    ax.axhspan(-0.1, 0.1, facecolor=band_rz, alpha=0.3, zorder=0)
    ax.axhline(0.4,  color=blues(0.65), linestyle="-", linewidth=1, alpha=0.5)
    ax.axhline(-0.4, color=blues(0.65), linestyle="-", linewidth=1, alpha=0.5)
    ax.axhline(0.1,  color=oranges(0.65), linestyle="-", linewidth=1, alpha=0.5)
    ax.axhline(-0.1, color=oranges(0.65), linestyle="-", linewidth=1, alpha=0.5)
    
    # --- Data lines: distinct per series with styles/markers ---
    ax.scatter(data['seq'],
        data['x_gradient'] * 8.4,
        label="X (×8.4)", color=c_x, linewidth=2.0, linestyle="-",
        s=0.5, zorder=3
    )
    ax.scatter(data['seq'],
        data['y_gradient'] * 8.4,
        label="Y (×8.4)", color=c_y, linewidth=2.0, linestyle="--",
        s=0.5, zorder=3
    )
    ax.scatter(data['seq'],
        data['radial_gradient'] * 4.2,
        label="Radial (×4.2)", color=c_r, linewidth=2.0, linestyle="-",
        s=0.5, zorder=4
    )
    ax.scatter(data['seq'],
        data['z_gradient'],
        label="Z", color=c_z, linewidth=2.0, linestyle="--",
        s=0.5, zorder=4
    )
    
    
    # --- Legends: one for data lines, one for bands (with matching colors) ---
    data_leg = ax.legend(bbox_to_anchor=(0.0, 1.25), loc='upper left',
                frameon=True, ncol=2, markerscale=4)
    ax.add_artist(data_leg)
    
    band_handles = [
        Patch(facecolor=band_xy, alpha=0.6, label="X/Y limit band (±0.4)"),
        Patch(facecolor=band_rz, alpha=0.6, label="Radial/Z limit band (±0.1)"),
    ]
    ax.legend(handles=band_handles,
              loc="upper right", frameon=True, bbox_to_anchor=(1.0, 1.25))
    
    # --- Labels, grid, cosmetics ---
    
    ax.set_ylim(-0.6, 0.6)
    return


In [ ]:
import logging

from astropy.time import Time, TimeDelta
from lsst.obs.lsst import LsstCam
from lsst.summit.utils import (
    ConsDbClient,
    getAirmassSeeingCorrection,
    getBandpassSeeingCorrection,
)
from lsst.summit.utils.efdUtils import (
    getEfdData,
    getMostRecentRowWithDataBefore,
    makeEfdClient,
)
from lsst.ts.ofc import BendModeToForce, OFCData, StateEstimator

from tqdm import tqdm as tqdm

__all__ = ["AOSDatabase"]


class AOSDatabase:
    table: pd.DataFrame

    def __init__(
        self,
        day_obs: int = 20250415,
        seq_min: int = 1,
        seq_max: int = 9999,
        consdb_url: str = "http://consdb-pq.consdb:8080/consdb",
    ) -> None:
        """Create fetcher.

        Parameters
        ----------
        seq_max : int, optional
            Maximum sequence number to fetch. Default is 9999.
        consdb_url : str, optional
            URL to create ConsDB client.
            The default is "http://consdb-pq.consdb:8080/consdb".
        """
        self.log = logging.getLogger(__name__)

        self.efd_client = makeEfdClient()
        self.cdb_client = ConsDbClient(consdb_url)

        self.det_order = list([191, 195, 199, 203])
        camera = LsstCam().getCamera()
        self.detector_names = [
            camera.get(det_id).getName() for det_id in self.det_order
        ]

        self.day_obs = day_obs
        self.seq_max = seq_max
        self.seq_min = seq_min
        self.table = pd.DataFrame()

        self.time_window = TimeDelta(0.2, format="sec")
        self.temp_time_window = TimeDelta(0.2, format="sec")

        self.ofc_data = OFCData("lsst")
        self.m2_bmf = BendModeToForce("M2", self.ofc_data)
        self.m1m3_bmf = BendModeToForce("M1M3", self.ofc_data)

    async def create(self, simplified=False):
        self.table = await self._fetch(
            self.day_obs, self.seq_min, self.seq_max, simplified
        )

    async def update(self, simplified: bool = False) -> None:
        """Update the database by grabbing more recent exposures.
        This will only grab new sequences, not re-fetch existing ones.
        """
        # First grab new sequences
        seq_min = self.table["seq"].max() + 1
        updated_table = await self._fetch(
            self.day_obs, seq_min, self.seq_max, simplified=simplified
        )
        self.table = pd.concat(
            [self.table, updated_table],
            ignore_index=True,
        )

    async def _fetch(
        self, day_obs: int, seq_min: int, seq_max: int, simplified: bool = False
    ) -> pd.DataFrame:
        query = f"""
            SELECT
            e.air_temp AS air_temp,
            e.airmass AS airmass,
            e.dimm_seeing AS dimm,
            e.altitude AS elevation,
            e.azimuth AS azimuth,
            e.exposure_id AS visit_id,
            e.physical_filter as band,
            e.day_obs AS day_obs,
            e.exp_midpt AS time,
            e.dimm_seeing AS seeing,
            e.seq_num AS seq,
            e.science_program AS block,
            ccdvisit1_quicklook.psf_sigma,
            ccdvisit1_quicklook.z4,
            ccdvisit1_quicklook.z5,
            ccdvisit1_quicklook.z6,
            ccdvisit1_quicklook.z7,
            ccdvisit1_quicklook.z8,
            ccdvisit1_quicklook.z9,
            ccdvisit1_quicklook.z10,
            ccdvisit1_quicklook.z11,
            ccdvisit1_quicklook.z12,
            ccdvisit1_quicklook.z13,
            ccdvisit1_quicklook.z14,
            ccdvisit1_quicklook.z15,
            ccdvisit1_quicklook.z16,
            ccdvisit1_quicklook.z17,
            ccdvisit1_quicklook.z18,
            ccdvisit1_quicklook.z19,
            ccdvisit1_quicklook.z20,
            ccdvisit1_quicklook.z21,
            ccdvisit1_quicklook.z22,
            ccdvisit1_quicklook.z23,
            ccdvisit1_quicklook.z24,
            ccdvisit1_quicklook.z25,
            ccdvisit1_quicklook.z26,
            ccdvisit1_quicklook.z27,
            ccdvisit1_quicklook.z28,
            ccdvisit1.detector as detector,
            q.psf_sigma_median AS psf_fwhm,
            q.aos_fwhm AS aos_fwhm,
            q.donut_blur_fwhm AS donut_blur_fwhm,
            q.physical_rotator_angle AS rotation_angle,
            e.obs_end,
            e.obs_start
            FROM
            cdb_lsstcam.ccdvisit1_quicklook AS ccdvisit1_quicklook,
            cdb_lsstcam.ccdvisit1 AS ccdvisit1,
            cdb_lsstcam.visit1 AS visit1,
            cdb_lsstcam.visit1_quicklook AS q,
            cdb_lsstcam.exposure AS e
            WHERE
            ccdvisit1.detector IN (191, 192, 195, 196, 199, 200, 203, 204)
            AND ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
            AND ccdvisit1.visit_id = visit1.visit_id
            AND ccdvisit1.visit_id = q.visit_id
            AND ccdvisit1.visit_id = e.exposure_id
            AND (e.img_type = 'science' or e.img_type = 'acq' or e.img_type = 'engtest')
            AND e.day_obs = {day_obs}
            AND (e.seq_num BETWEEN {seq_min} AND {seq_max})
            AND e.airmass > 0
            AND e.band != 'none'
        """
        self.table = self.cdb_client.query(query).to_pandas()

        # Correctly declare aos_fwhm and donut_blur_fwhm as float
        self.table['aos_fwhm'] = pd.to_numeric(self.table['aos_fwhm'])
        self.table['donut_blur_fwhm'] = pd.to_numeric(self.table['donut_blur_fwhm'])

        # Convert PSF sigma to FWHM
        sig2fwhm = 2 * np.sqrt(2 * np.log(2))
        pixel_scale = 0.2  # arcsec / pixel
        self.table["psf_fwhm"] = self.table["psf_fwhm"] * sig2fwhm * pixel_scale

        self.table["fwhm_zenith_500nm"] = [
            fwhm
            * getAirmassSeeingCorrection(airmass)
            * getBandpassSeeingCorrection(band)
            for fwhm, band, airmass in zip(
                self.table["psf_fwhm"], self.table["band"], self.table["airmass"]
            )
        ]

        zernike_columns = [f"z{i}" for i in range(4, 29)]
        self.table["zernikes"] = self.table[zernike_columns].apply(
            lambda row: np.array(row.fillna(0.0).values, dtype=float), axis=1
        )
        self.table["zernikes_fwhm"] = self.table["zernikes"].apply(
            convertZernikesToPsfWidth
        )
        
        # Get the data for the science CCDs for fwhm_05 and fwhm_95
        visits_query = f'''
        SELECT 
        ccdvisit1_quicklook.psf_sigma,
        ccdvisit1.detector,
        visit1.visit_id,
        visit1.seq_num AS seq,
        visit1.day_obs,
        visit1.airmass
        FROM
        cdb_lsstcam.ccdvisit1_quicklook AS ccdvisit1_quicklook,
        cdb_lsstcam.ccdvisit1 AS ccdvisit1,
        cdb_lsstcam.visit1_quicklook AS visit1_quicklook,
        cdb_lsstcam.visit1 AS visit1 
        WHERE 
        ccdvisit1.ccdvisit_id = ccdvisit1_quicklook.ccdvisit_id
        AND ccdvisit1.visit_id = visit1.visit_id 
        AND visit1.visit_id = visit1_quicklook.visit_id
        AND ccdvisit1.detector NOT IN (168, 188, 123, 27, 0, 20, 65, 161)
        AND visit1.airmass > 0
        AND visit1.day_obs = {self.day_obs}
        AND (visit1.seq_num BETWEEN {self.seq_min} AND {self.seq_max})
        AND (visit1.img_type = 'science' or visit1.img_type = 'acq' or visit1.img_type = 'engtest')
        '''
        
        ccdvisits = self.cdb_client.query(visits_query).to_pandas()
        ccdvisits["psf_fwhm"] = ccdvisits["psf_sigma"] * sig2fwhm * pixel_scale
        ccdvisits["psf_fwhm"] = pd.to_numeric(ccdvisits["psf_fwhm"], errors="coerce")
        groups = ccdvisits.groupby('visit_id')
        visits_summary = pd.DataFrame({
            'day_obs': groups['day_obs'].first(),
            'seq': groups['seq'].median(),
            'psf_fwhm_05': groups['psf_fwhm'].quantile(0.05),
            'psf_fwhm_95': groups['psf_fwhm'].quantile(0.95),
        })
        visits_summary['psf_fwhm_95_05'] = np.sqrt(visits_summary['psf_fwhm_95']**2 - visits_summary['psf_fwhm_05']**2)
        self.table = pd.merge(
                        self.table, visits_summary, how="left", on=["seq", "day_obs"])
        
        self.table = await find_faults(self.efd_client, self.table)
        
        if not simplified:
            unique_day_seq = (
                self.table[["day_obs", "seq", "obs_end", "obs_start"]]
                .drop_duplicates()
                .reset_index(drop=True)
            )
            (
                inside_air_temp,
                days,
                seqs,
                lut,
                cam_air_temp,
                states,
                above_m1m3_temp,
                outside_temp,
                m2_temp,
                correction_seq,
            ) = ([] for _ in range(10))
            for idx, row in tqdm(unique_day_seq.iterrows(), total=len(unique_day_seq), disable=True):
                day_obs = int(row["day_obs"])
                seq = int(row["seq"])

                rec_end = row["obs_end"]
                rec_start = row["obs_start"]

                # ---------- Environment variables -------------
                # ----------------------------------------------
                # Get state
                try:
                    total_lut_gravity = [f"lutGravity{i}" for i in range(72)]
                    total_lut_temperature = [f"lutTemperature{i}" for i in range(72)]
                    m2_actuator_data = await self.efd_client.select_time_series(
                        "lsst.sal.MTM2.axialForce",
                        ["*"],
                        Time(rec_start, scale="utc"),
                        Time(rec_start, scale="utc") + self.time_window,
                        convert_influx_index=True
                    )
    
                    m2_combined_lut = (
                        m2_actuator_data[total_lut_gravity].values
                        + m2_actuator_data[total_lut_temperature].values
                    )
                    m2_combined_lut = m2_combined_lut.mean(axis=0)
                    m2_dofs_lut = self.m2_bmf.bending_mode(m2_combined_lut)
                except Exception:
                    m2_dofs_lut = np.full(20, np.nan)

                try:
                    z_cols = [f"zForces{i}" for i in range(156)]
                    m1m3_el_lut = await self.efd_client.select_time_series(
                        "lsst.sal.MTM1M3.appliedElevationForces",
                        z_cols,
                        Time(rec_start, scale="utc"),
                        Time(rec_start, scale="utc") + self.time_window,
                        convert_influx_index=True
                    )
                    m1m3_el_lut = m1m3_el_lut[z_cols].values.mean(axis=0)

                    m1m3_az_lut = await self.efd_client.select_time_series(
                        "lsst.sal.MTM1M3.appliedAzimuthForces",
                        z_cols,
                        Time(rec_start, scale="utc"),
                        Time(rec_start, scale="utc") + self.time_window,
                        convert_influx_index=True
                    )
                    m1m3_az_lut = m1m3_az_lut[z_cols].values.mean(axis=0)

                    m1m3_temp_lut = await self.efd_client.select_time_series(
                        "lsst.sal.MTM1M3.appliedThermalForces",
                        z_cols,
                        Time(rec_start, scale="utc"),
                        Time(rec_start, scale="utc") + self.time_window,
                        convert_influx_index=True
                    )
                    m1m3_temp_lut = m1m3_temp_lut[z_cols].values.mean(axis=0)

                    # Align the three DataFrames
                    # (assumes same shape/timestamps)
                    m1m3_combined_lut = m1m3_el_lut + m1m3_az_lut + m1m3_temp_lut
                    m1m3_combined_lut = m1m3_combined_lut
                    m1m3_dofs_lut = self.m1m3_bmf.bending_mode(m1m3_combined_lut)
                except Exception:
                    m1m3_dofs_lut = np.full(20, np.nan)
                try:                    
                    cam_hexapod_data = getMostRecentRowWithDataBefore(
                        self.efd_client,
                        "lsst.sal.MTHexapod.logevent_compensationOffset",
                        Time(rec_end, scale="utc"),
                        maxSearchNMinutes=10,
                        where=lambda df: df["salIndex"] == 1,
                    )

                    m2_hexapod_data = getMostRecentRowWithDataBefore(
                        self.efd_client,
                        "lsst.sal.MTHexapod.logevent_compensationOffset",
                        Time(rec_end, scale="utc"),
                        maxSearchNMinutes=10,
                        where=lambda df: df["salIndex"] == 2,
                    )

                    hexapod_val = np.array(
                        [
                            m2_hexapod_data["z"],
                            m2_hexapod_data["x"],
                            m2_hexapod_data["y"],
                            m2_hexapod_data["u"],
                            m2_hexapod_data["v"],
                            cam_hexapod_data["z"],
                            cam_hexapod_data["x"],
                            cam_hexapod_data["y"],
                            cam_hexapod_data["u"],
                            cam_hexapod_data["v"],
                        ]
                    )
                    lut_val = np.concatenate([hexapod_val, m1m3_dofs_lut, m2_dofs_lut])
                except Exception:
                    lut_val = np.full(50, np.nan)

                event = getMostRecentRowWithDataBefore(
                    self.efd_client,
                    "lsst.sal.MTAOS.logevent_degreeOfFreedom",
                    timeToLookBefore=Time(rec_start, scale="utc"),
                )
                out = np.empty(
                    50,
                )
                for i in range(50):
                    out[i] = event[f"aggregatedDoF{i}"]
                states_val = out

                seq_num_corr = event["visitId"]
                
                # Get outside temperature
                temp_outside_data = await self.efd_client.select_time_series(
                    "lsst.sal.ESS.temperature",
                    ["temperatureItem0"],
                    Time(rec_start, scale="utc"),
                    Time(rec_end, scale="utc") + self.temp_time_window,
                    index=301,
                    convert_influx_index=True
                )
                if "temperatureItem0" in temp_outside_data:
                    outside_temp_val = temp_outside_data["temperatureItem0"].mean()
                else:
                    outside_temp_val = np.nan

                # Get M2 temperature
                m2_temp_data = await self.efd_client.select_time_series(
                    "lsst.sal.MTM2.temperature",
                    ["ring6"],
                    Time(rec_start, scale="utc"),
                    Time(rec_end, scale="utc") + self.temp_time_window,
                )
                if "ring6" in m2_temp_data:
                    m2_temp_val = m2_temp_data["ring6"].mean()
                else:
                    m2_temp_val = np.nan

                # Get cam temperature
                cam_temp_data = await self.efd_client.select_time_series(
                    "lsst.sal.ESS.temperature",
                    ["temperatureItem0"],
                    Time(rec_start, scale="utc"),
                    Time(rec_end, scale="utc") + self.temp_time_window,
                    index=111,
                )
                if "temperatureItem0" in cam_temp_data:
                    cam_air_temp_val = cam_temp_data["temperatureItem0"].mean()
                else:
                    cam_air_temp_val = np.nan

                # Get temperature above m1m3
                temp_m1m3_data = await self.efd_client.select_time_series(
                    "lsst.sal.ESS.temperature",
                    ["temperatureItem0"],
                    Time(rec_start, scale="utc"),
                    Time(rec_end, scale="utc") + self.temp_time_window,
                    index=113,
                    convert_influx_index=True
                )
                if "temperatureItem0" in temp_m1m3_data:
                    above_m1m3_temp_val = temp_m1m3_data["temperatureItem0"].mean()
                else:
                    above_m1m3_temp_val = np.nan

                # Get inside dome air temperature
                inside_air_data = await self.efd_client.select_time_series(
                    "lsst.sal.ESS.temperature",
                    ["temperatureItem0"],
                    Time(rec_start, scale="utc"),
                    Time(rec_end, scale="utc") + self.temp_time_window,
                    index=112,
                    convert_influx_index=True
                )
                if "temperatureItem0" in inside_air_data:
                    inside_air_temp_val = inside_air_data["temperatureItem0"].mean()
                else:
                    inside_air_temp_val = np.nan

                lut.append(lut_val)
                above_m1m3_temp.append(above_m1m3_temp_val)
                inside_air_temp.append(inside_air_temp_val)
                cam_air_temp.append(cam_air_temp_val)
                m2_temp.append(m2_temp_val)
                outside_temp.append(outside_temp_val)
                states.append(states_val)
                days.append(day_obs)
                seqs.append(seq)
                correction_seq.append(seq_num_corr)

            # Calculate FWHM Zernike contributions
            efd_table = pd.DataFrame(
                {
                    "day_obs": days,
                    "seq": seqs,
                    "inside_dome_air_temp": inside_air_temp,
                    "cam_air_temp": cam_air_temp,
                    "m2_temp": m2_temp,
                    "above_m1m3_temp": above_m1m3_temp,
                    "outside_temp": outside_temp,
                    "lut_state": lut,
                    "dof_state": states,
                    "seq_num_corr": correction_seq,
                }
            )

            self.table = pd.merge(
                self.table, efd_table, how="left", on=["seq", "day_obs"]
            )
            m1m3_gradient_table = await get_m1m3_gradients(self.efd_client, unique_day_seq)
            self.table = pd.merge(
                self.table, m1m3_gradient_table, how="left", on=["seq", "day_obs"]
            )
            self.table["m2_delta_t"] = (
                self.table["m2_temp"] - self.table["inside_dome_air_temp"]
            )
            self.table["dome_delta_t"] = (
                self.table["outside_temp"] - self.table["inside_dome_air_temp"]
            )
            self.table["cam_m1m3_delta_t"] = (
                self.table["cam_air_temp"] - self.table["above_m1m3_temp"]
            )

        return self.table


In [ ]:

zk_groups = [[0], [11 - 4], [1, 2], [3,4], [5, 6], [22]]
zk_group_labels = ['Z4', 'Z11', 'Z5 / Z6', 'Z7 / Z8', ' Z9 / Z10', 'Z22']

groups = [[0], [5], [1, 2, 6, 7], [3, 4], [8,9]]
group_labels = [' M2 dz', 'Cam dz', 'Decenters', 'M2 tilts', 'Cam tilts']
labels = ['m2 dz', 'm2 dx', 'm2 dy', 'm2 rx', 'm2 ry',
          'cam dz', 'cam dx', 'cam dy', 'cam rx', 'cam ry']


mirror_groups = [[10, 11, 30, 31], [12, 34], [13, 14, 32, 33], [15, 16, 35, 36], [17, 18, 37, 38]]
mirror_group_labels = ['Astig', 'Spherical', 'Trefoil', 'Coma', 'Quad']
all_labels = ['M2 dz', 'M2 dx', 'M2 dy', 'M2 rx', 'M2 ry',
     'cam dz', 'cam dx', 'cam dy', 'cam rx', 'cam ry',
     '$B_{{1,1}}$', '$B_{{1,2}}$', '$B_{{1,3}}$', '$B_{{1,4}}$', '$B_{{1,5}}$',
     '$B_{{1,6}}$', '$B_{{1,7}}$', '$B_{{1,8}}$', '$B_{{1,9}}$', '$B_{{1,10}}$',
     '$B_{{1,11}}$', '$B_{{1,12}}$', '$B_{{1,13}}$', '$B_{{1,14}}$', '$B_{{1,15}}$',
     '$B_{{1,16}}$', '$B_{{1,17}}$', '$B_{{1,18}}$', '$B_{{1,19}}$', '$B_{{1,20}}$',
     '$B_{{2,1}}$', '$B_{{2,2}}$', '$B_{{2,3}}$', '$B_{{2,4}}$', '$B_{{2,5}}$',
     '$B_{{2,6}}$', '$B_{{2,7}}$', '$B_{{2,8}}$', '$B_{{2,9}}$', '$B_{{2,10}}$',
     '$B_{{2,11}}$', '$B_{{2,12}}$', '$B_{{2,13}}$', '$B_{{2,14}}$', '$B_{{2,15}}$',
     '$B_{{2,16}}$', '$B_{{2,17}}$', '$B_{{2,18}}$', '$B_{{2,19}}$', '$B_{{2,20}}$'
     ]

## Simplified Nightly Report

In [ ]:
import os 
#os.environ["no_proxy"] += ",.consdb"
db = AOSDatabase(day_obs=day_obs, seq_min=seq_min, seq_max=seq_max)
await db.create(simplified=True)
table = db.table
print(table['rotation_angle'].dtypes)

In [ ]:
if len(table)>0:
    filtered_table = table[table['day_obs'] == day_obs]
    #filtered_table['rotation_angle'] = filtered_table['rotation_angle'].astype(float) # Had to add this line ??
    raw_filtered_table = table[table['day_obs'] == day_obs]
    filtered_table = filtered_table.select_dtypes(include="number")
    filtered_table['band'] = raw_filtered_table['band']
    filtered_table = filtered_table.groupby("seq").agg({col: 'first' if col == 'band' else 'mean' for col in filtered_table.columns})
    
    # -- AOS jumps
    aos_diff = filtered_table["aos_fwhm"].diff()
    jump_indices = filtered_table["seq"][aos_diff > 0.3].tolist()
    
    # -- States array and labels
    states_per_seq = (
        raw_filtered_table[["seq", "zernikes_fwhm"]]
        .drop_duplicates("seq")
        .dropna(subset=["zernikes_fwhm"])
        .set_index("seq")
    )
    
    
    zernikes_fwhm = np.vstack(states_per_seq["zernikes_fwhm"].values)
    seqs = states_per_seq.index.values
else:
    print(f'No  data available for {day_obs},  seq_nums {seq_min}-{seq_max}')

In [ ]:
if len(table)>0:
    fig = plt.figure(figsize=(30, 15))

    linewidth = 0.7
    # Top 5x4 GridSpec occupies the upper half
    gs_top = gridspec.GridSpec(
        nrows=5, ncols=4,
        width_ratios=[4, 2, 2, 2],
        height_ratios=[1]*5,
        hspace=0.0,
        wspace=0.25,
        top=0.97,
        bottom=0.54  # ends halfway down
    )
    
    # Bottom 5x4 GridSpec occupies the lower half
    gs_bot = gridspec.GridSpec(
        nrows=5, ncols=4,
        width_ratios=[4, 2, 2, 2],
        height_ratios=[1]*5,
        hspace=0.0,
        top=0.46,
        bottom=0.05
    )
    
    # -- Left column: Survey performance
    axes = []
    for i in range(5):
        ax = fig.add_subplot(gs_top[i, 0], sharex=axes[0] if i > 0 else None)
        axes.append(ax)
    axes[0].scatter(filtered_table['seq'], filtered_table['fwhm_zenith_500nm'], s=3, label='FWHM_Zenith_500nm')
    axes[0].scatter(filtered_table['seq'], filtered_table['donut_blur_fwhm'], s=3, label='Donut Blur FWHM')
    axes[0].scatter(filtered_table['seq'], filtered_table["psf_fwhm_95_05"], s=3, label="FWHM:sqrt(95^2-5^2)")
    try:
        axes[0].scatter(filtered_table['seq'], filtered_table['dimm'], s=3, label='DIMM')
    except KeyError:
        pass
    axes[0].legend(fontsize=8, bbox_to_anchor=(1.0, 1.5), loc='upper right')
    ymin0 = 0; ymax0 = 2.0
    axes[0].set_ylim(ymin0, ymax0)
    axes[0].text(0, ymax0 * 1.1, "... Faults", fontsize=14, color='magenta')
    axes[0].set_ylabel('FWHM [arcsec]')
    axes[0].set_title(f'Delivered Seeing and System Variables')
    axes[1].scatter(filtered_table['seq'], filtered_table["aos_fwhm"], s=3)
    axes[2].scatter(filtered_table['seq'], filtered_table['elevation'], color='k', s=3)
    axes[3].scatter(filtered_table['seq'], filtered_table['azimuth'], color='k', s=3)
    axes[4].scatter(filtered_table['seq'], filtered_table['rotation_angle'], color='k', s=3)
    
    axes[0].set_ylabel('FWHM\n[arcsec]')
    axes[1].set_ylabel('AOS FWHM\n[arcsec]')
    axes[2].set_ylabel('Elevation\n[deg]')
    axes[3].set_ylabel('Azimuth\n[deg]')
    axes[4].set_ylabel('Rotation\n[deg]')
    axes[4].set_xlabel('Sequence Number')
    
    annotate_bands(filtered_table, axes[4])
    axes[4].legend(ncols=3)
    
    
    for ax in axes[1::]:
        for x in jump_indices:
            ax.axvline(x=x, color='red', linestyle='--', linewidth=linewidth)
    # Add block boundaries and names
    blocks = find_blocks(table) # Find active blocks
    maxSeq = max(table['seq'].values)
    for i, changeSeq in enumerate(blocks['seq']):
        if i < len(blocks) - 1:
            changeWidth = blocks['seq'].iloc[i+1] - changeSeq
        else:
            changeWidth = maxSeq - changeSeq
        for ax in axes[0::]:
            ax.axvline(changeSeq, ls = '--', color='k', linewidth=linewidth, alpha=0.5)
        if changeWidth > 5:
            (ymin, ymax) = axes[1].get_ylim()
            ytext = ymin = (ymax - ymin) * 0.5
            axes[1].text(int(changeSeq + changeWidth / 2 - 2), ytext, 
                         blocks['block_after'].iloc[i], fontsize=8, rotation=90, alpha=0.6) 
    # Now plot the faults
    faults = table[table['faults'].notnull()] 
    for k in range(len(faults)):
        for ax in axes[0::]:
            ax.axvline(faults['seq'].iloc[k], color='magenta', ls=':')
        
    for ax in axes[:-1]:
        ax.tick_params(labelbottom=False)
    for ax in axes:
        ax.grid(True, alpha=0.5)
        ax.tick_params(direction='in', which='both')
    
    
    # -- Right column: DoF plots with shared x-axis (not y)
    axes = [fig.add_subplot(gs_top[i, 1]) for i in range(5)]
    for id_group, (ax, zk_group) in enumerate(zip(axes, zk_groups)):
        zk_labels = zk_group_labels[id_group]
        for zk_idx, i in enumerate(zk_group):
            if len(zk_group) == 1:
                zk_label = zk_labels
            if len(zk_group) == 2:
                zk_label = zk_labels.split('/')[zk_idx].strip()
            color = 'black' if zk_idx == 0 else 'gray' if zk_idx == 1 else None
            vals = zernikes_fwhm[:, i]
            ax.scatter(seqs, vals, s=5, color=color, label=zk_label)
        ax.set_ylabel(zk_group_labels[id_group])
        ax.grid(True, alpha=0.5)
        ax.tick_params(direction="in")
        ax.legend(bbox_to_anchor=(1.22, 0.5), loc='center right')
    
    for ax in axes[:-1]:
        ax.tick_params(labelbottom=False)
        ax.grid(True, alpha=0.5)
    for ax in axes:
        for x in jump_indices:
            ax.axvline(x=x, color='red', linestyle='--', linewidth=linewidth)
    axes[-1].set_xlabel("Sequence Number")
    axes[0].set_title(f'Optical Aberrations')
    
    fig.suptitle("Survey Mode Performance Simplified – Day " + str(day_obs), fontsize=18, x=0.35, y=1.03)
    
else:
    print('No data to plot')

## Full Nightly Report

In [ ]:
db = AOSDatabase(day_obs=day_obs, seq_min=seq_min, seq_max=seq_max)
await db.create(simplified=False)
table = db.table

In [ ]:
if len(table)>0:
    filtered_table = table[table['day_obs'] == day_obs]
    #filtered_table['rotation_angle'] = filtered_table['rotation_angle'].astype(float) # Had to add this line ??
    raw_filtered_table = table[table['day_obs'] == day_obs]
    filtered_table = filtered_table.select_dtypes(include="number")
    filtered_table['band'] = raw_filtered_table['band']
    filtered_table = filtered_table.groupby("seq").agg({col: 'first' if col == 'band' else 'mean' for col in filtered_table.columns})
    
    # -- AOS jumps
    aos_diff = filtered_table["aos_fwhm"].diff()
    jump_indices = filtered_table["seq"][aos_diff > 0.3].tolist()
    
    # -- States array and labels
    states_per_seq = (
        raw_filtered_table[["seq", "dof_state", "zernikes_fwhm", "lut_state"]]
        .drop_duplicates("seq")
        .dropna(subset=["dof_state", "zernikes_fwhm", "lut_state"])
        .set_index("seq")
    )
    
    dof_state = np.vstack(states_per_seq["dof_state"].values)
    zernikes_fwhm = np.vstack(states_per_seq["zernikes_fwhm"].values)
    lut_state = np.vstack(states_per_seq["lut_state"].values)
    seqs = states_per_seq.index.values
else:
    print(f'No  data available for {day_obs},  seq_nums {seq_min}-{seq_max}')
len(filtered_table['rotation_angle'])

In [ ]:
if len(table)>0:
    fig = plt.figure(figsize=(30, 15))
    
    linewidth = 0.7
    # Top 5x4 GridSpec occupies the upper half
    gs_top = gridspec.GridSpec(
        nrows=6, ncols=4,
        width_ratios=[4, 2, 2, 2],
        height_ratios=[1]*6,
        hspace=0.0,
        wspace=0.25,
        top=0.97,
        bottom=0.54  # ends halfway down
    )
    
    # Bottom 5x4 GridSpec occupies the lower half
    gs_bot = gridspec.GridSpec(
        nrows=5, ncols=4,
        width_ratios=[4, 2, 2, 2],
        height_ratios=[1]*5,
        hspace=0.0,
        wspace=0.25,
        top=0.46,
        bottom=0.05
    )
    
    # -- Left column: Survey performance
    axes = []
    for i in range(6):
        ax = fig.add_subplot(gs_top[i, 0], sharex=axes[0] if i > 0 else None)
        axes.append(ax)
    axes[0].scatter(filtered_table['seq'], filtered_table['fwhm_zenith_500nm'], s=3, label='FWHM_Zenith_500nm')
    axes[0].scatter(filtered_table['seq'], filtered_table['donut_blur_fwhm'], s=3, label='Donut Blur FWHM')
    axes[0].scatter(filtered_table['seq'], filtered_table["psf_fwhm_95_05"], s=3, label="FWHM:sqrt(95^2-5^2)")
    try:
        axes[0].scatter(filtered_table['seq'], filtered_table['dimm'], s=3, label='DIMM')
    except KeyError:
        pass
    axes[0].legend(fontsize=8, bbox_to_anchor=(1.0, 1.6), loc='upper right')
    ymin0 = 0; ymax0 = 2.0
    axes[0].set_ylim(ymin0, ymax0)
    axes[0].text(0, ymax0 * 1.1, "... Faults", fontsize=14, color='magenta')
    axes[0].set_ylabel('FWHM [arcsec]')
    axes[0].set_title(f'Delivered Seeing and System Variables')
    axes[1].scatter(filtered_table['seq'], filtered_table["aos_fwhm"], s=3, label='AOS FWHM')
    axes[2].scatter(filtered_table['seq'], filtered_table['elevation'], color='k', s=3)
    axes[3].scatter(filtered_table['seq'], filtered_table['azimuth'], color='k', s=3)
    axes[4].scatter(filtered_table['seq'], filtered_table['rotation_angle'], color='k', s=3)
    max_corr_lag = 10
    corr_lag = filtered_table['seq'] - filtered_table['seq_num_corr']
    corr_lag = np.clip(corr_lag, 0, max_corr_lag)
    axes[5].scatter(filtered_table['seq'], corr_lag, color='k', s=3)
    
    axes[0].set_ylabel('FWHM\n[arcsec]')
    axes[1].set_ylabel('AOS FWHM\n[arcsec]')
    axes[2].set_ylabel('Elevation\n[deg]')
    axes[3].set_ylabel('Azimuth\n[deg]')
    axes[4].set_ylabel('Rotator\n[deg]')
    axes[5].set_ylabel('Correction Lag\n[# of visits]')
    axes[5].set_xlabel('Sequence Number')
    
    annotate_bands(filtered_table, axes[5])
    axes[5].legend(ncols=3, fontsize=8)
    
    
    for ax in axes[1::]:
        for x in jump_indices:
            ax.axvline(x=x, color='red', linestyle='--', linewidth=linewidth)
    # Add block boundaries and names
    blocks = find_blocks(table) # Find active blocks
    maxSeq = max(table['seq'].values)
    for i, changeSeq in enumerate(blocks['seq']):
        if i < len(blocks) - 1:
            changeWidth = blocks['seq'].iloc[i+1] - changeSeq
        else:
            changeWidth = maxSeq - changeSeq
        for ax in axes[0::]:
            ax.axvline(changeSeq, ls = '--', color='k', linewidth=linewidth, alpha=0.5)
        if changeWidth > 5:
            (ymin1, ymax1) = axes[1].get_ylim()
            ytext = ymin1 = (ymax1 - ymin1) * 0.5
            axes[1].text(int(changeSeq + changeWidth / 2 - 2), ytext, 
                         blocks['block_after'].iloc[i], fontsize=8, rotation=90, alpha=0.6)    

    # Now plot the faults
    faults = table[table['faults'].notnull()] 
    for k in range(len(faults)):
        for ax in axes[0::]:
            ax.axvline(faults['seq'].iloc[k], color='magenta', ls=':')

    for ax in axes[:-1]:
        ax.tick_params(labelbottom=False)
    for ax in axes:
        ax.grid(True, alpha=0.5)
        ax.tick_params(direction='in', which='both')
    
    # Thermal gradients 
    axes[0] = fig.add_subplot(gs_bot[0:2, 0])
    for i in range(2,5):
        axes[i] = fig.add_subplot(gs_bot[i, 0])
    plot_m1m3_gradients(filtered_table, axes[0])
    axes[2].scatter(filtered_table['seq'], filtered_table['m2_delta_t'], color='k', s=3)
    axes[3].scatter(filtered_table['seq'], filtered_table['cam_m1m3_delta_t'], color='k', s=3)
    axes[4].scatter(filtered_table['seq'], filtered_table['dome_delta_t'], color='k', s=3)
    
    axes[0].set_ylabel(f'M1M3 Gradients\n[C]')
    axes[2].set_ylabel('M2 - Dome\n[C]')
    axes[3].set_ylabel('Cam - M1M3\n[C]')
    axes[4].set_ylabel('Out - Dome\n[C]')
    axes[4].set_xlabel('Sequence Number')
    
    for ax in axes:
        for x in jump_indices:
            ax.axvline(x=x, color='red', linestyle='--', linewidth=linewidth)
    for ax in axes[:-1]:
        ax.tick_params(labelbottom=False)
    for ax in axes:
        ax.grid(True, alpha=0.5)
        ax.tick_params(direction='in', which='both')
    axes[0].set_title(f'Thermal Gradients')
    
    # -- Right column: DoF plots with shared x-axis (not y)
    axes = [fig.add_subplot(gs_top[i, 1]) for i in range(6)]
    for id_group, (ax, zk_group) in enumerate(zip(axes, zk_groups)):
        zk_labels = zk_group_labels[id_group]
        for zk_idx, i in enumerate(zk_group):
            if len(zk_group) == 1:
                zk_label = zk_labels
            if len(zk_group) == 2:
                zk_label = zk_labels.split('/')[zk_idx].strip()
            color = 'black' if zk_idx == 0 else 'gray' if zk_idx == 1 else None
            vals = zernikes_fwhm[:, i]
            ax.scatter(seqs, vals, s=5, color=color, label=zk_label)
        ax.set_ylabel(zk_group_labels[id_group])
        ax.grid(True, alpha=0.5)
        ax.tick_params(direction="in")
        ax.legend(bbox_to_anchor=(1.22, 0.5), loc='center right', frameon=False)
    
    for ax in axes[:-1]:
        ax.tick_params(labelbottom=False)
        ax.grid(True, alpha=0.5)
    for ax in axes:
        for x in jump_indices:
            ax.axvline(x=x, color='red', linestyle='--', linewidth=linewidth)
    axes[-1].set_xlabel("Sequence Number")
    axes[0].set_title(f'Optical Aberrations')
    
    
    # -- Right column: DoF plots with shared x-axis (not y)
    axes = [fig.add_subplot(gs_bot[i, 1]) for i in range(5)]
    for id_group, (ax, dof_group) in enumerate(zip(axes, groups)):
        for i in dof_group:
            vals = (lut_state[:, i] + dof_state[:, i])
            ax.scatter(seqs, vals, label=f"{labels[i]}", s=3)
        ax.set_ylabel(group_labels[id_group])
        ax.grid(True, alpha=0.5)
        ax.tick_params(direction="in")
        ax.legend(bbox_to_anchor=(1.28, 0.5), loc='center right', frameon=False)
    
    for ax in axes[:-1]:
        ax.tick_params(labelbottom=False)
        ax.grid(True, alpha=0.5)
    for ax in axes:
        for x in jump_indices:
            ax.axvline(x=x, color='red', linestyle='--', linewidth=linewidth)
    axes[0].set_title(f'Hexapods State (LUT + trim)')
    axes[-1].set_xlabel("Sequence Number")
    
    my_suptitle = fig.suptitle("Survey Mode Performance and AOS DoFs – Day " + str(day_obs), fontsize=18, x=0.35, y=1.03)
else:
    print('No data to plot')
fig.savefig(f"/home/cslage/MTAOS/times_square_notebooks/Full_Night_Report_{day_obs}.png", bbox_inches='tight', pad_inches=1.2, bbox_extra_artists=[my_suptitle])

In [ ]:
filtered_table.columns

In [ ]:
dof_state

In [ ]:
test = dof_state[(dof_state['seq']>58) & (dof_state['seq']<=62)]

In [ ]:
seq_1 = 291
seq_2 = 292

for seq, dof in zip(seqs, dof_state):
    if seq == seq_1:
        dof_1 = dof
    if seq == seq_2:
        dof_2 = dof
for j in range(50):
    if j in [1,2,3,4,6,7,8,9]:
        print(f"seq {seq_1} {all_labels[j]} = {dof_1[j]:.4f} \t \t \t seq {seq_2} {all_labels[j]} = {dof_2[j]:.4f}")
    else:
        print(f"seq {seq_1} {all_labels[j]} = {dof_1[j]:.4f} \t \t seq {seq_2} {all_labels[j]} = {dof_2[j]:.4f}")

In [ ]:
seq_1 =194
seq_2 =195

for seq, lut in zip(seqs, lut_state):
    if seq == seq_1:
        lut_1 = lut
    if seq == seq_2:
        lut_2 = lut
for j in range(50):
    if j in [3,4,8,9]:
        print(f"seq {seq_1} {all_labels[j]} = {lut_1[j]:.4f} \t \t \t seq {seq_2} {all_labels[j]} = {lut_2[j]:.4f}")
    else:
        print(f"seq {seq_1} {all_labels[j]} = {lut_1[j]:.4f} \t \t seq {seq_2} {all_labels[j]} = {lut_2[j]:.4f}")

In [ ]:
all_labels[17]